In [75]:
sampling_kwargs = {'temperature': 1.0, 'top_p': 0.3, 'freq_penalty': 1.0}

In [76]:
sampling_kwargs

{'temperature': 1.0, 'top_p': 0.3, 'freq_penalty': 1.0}

In [1]:
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_act_name

In [2]:
model = HookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [29]:
prompt = "I hate you because "

In [30]:
steer_positive_prompt = "Bread"
steer_negative_prompt = "  "

In [31]:
tokens = model.to_tokens(prompt)
steer_positive_tokens = model.to_tokens(steer_positive_prompt)
steer_negative_tokens = model.to_tokens(steer_negative_prompt)

In [32]:
steer_positive_tokens

tensor([[50256,    33,   961]])

In [33]:
steer_negative_tokens

tensor([[50256,   220,   220]])

In [8]:
hook_name = "blocks.6.hook_resid_pre"

In [44]:
_, positive_activations = model.run_with_cache(steer_positive_tokens)
_, negative_activations = model.run_with_cache(steer_negative_tokens)

In [45]:
target_positive_activations = positive_activations[hook_name] * 5
target_negative_activations = negative_activations[hook_name] * 5

In [43]:
target_positive_activations.shape

torch.Size([1, 3, 768])

In [46]:
target_negative_activations.shape

torch.Size([1, 3, 768])

In [48]:
activations_seq_len = target_positive_activations.shape[1]

In [50]:
sequence_slice = slice(0, activations_seq_len)

In [52]:
res_stream_slice = slice(None, None, None)

In [57]:
n_positive_tokens = steer_positive_tokens.shape[-1]

In [63]:
from IPython.core.debugger import set_trace

In [71]:
def add_resid_pre_activations(activations, hook):
    # set_trace()
    activations[:, :n_positive_tokens, :] += target_positive_activations * 3
    return activations

In [72]:
model.reset_hooks()

In [73]:
model.add_hook(hook_name, add_resid_pre_activations)

In [74]:
generated_tokens = model.generate(tokens, max_new_tokens=20)

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: output with shape [1, 1, 768] doesn't match the broadcast shape [1, 3, 768]